
# SOCIODEMOGRAPHIC

In [67]:
import numpy as np 
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

# Cargar el archivo parquet
df_full = pd.read_parquet("C:/Users/Bobby/Desktop/Master Data Analysis/TFM/easyMoney_fintech/data/df_completo.parquet")
# eliminamos columnas que no aportan información
df_full = df_full.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y', 'Unnamed: 0'], axis=1)

# Ordenar df_full por pk_cid y pk_partition
df_full = df_full.sort_values(by=['pk_cid', 'pk_partition'])

# modificamos el tipo de dato de las columnas, SIMPLEMENTE CAMBIA EL TIPO DE DATO
df_full["pk_partition"] = pd.to_datetime(df_full["pk_partition"])
df_full["entry_date"] = pd.to_datetime(df_full["entry_date"], errors='coerce')
df_full["entry_channel"] = df_full["entry_channel"].astype('category')
df_full['active_customer'] = df_full['active_customer'].astype('int32')
df_full['segment'] = df_full['segment'].astype('category')
df_full['short_term_deposit'] = df_full['short_term_deposit'].astype('int32')
df_full['loans'] = df_full['loans'].astype('int32')
df_full['mortgage'] = df_full['mortgage'].astype('int32')
df_full['funds'] = df_full['funds'].astype('int32')
df_full['securities'] = df_full['securities'].astype('int32')
df_full['long_term_deposit'] = df_full['long_term_deposit'].astype('int32')
df_full['credit_card'] = df_full['credit_card'].astype('int32')
# df_full['payroll'] = df_full['payroll'].astype('int32')
# df_full['pension_plan'] = df_full['pension_plan'].astype('int32')
df_full['payroll_account'] = df_full['payroll_account'].astype('int32')
df_full['emc_account'] = df_full['emc_account'].astype('int32')
df_full['debit_card'] = df_full['debit_card'].astype('int32')
df_full['em_account_p'] = df_full['em_account_p'].astype('int32')
df_full['em_acount'] = df_full['em_acount'].astype('int32')
df_full["country_id"] = df_full["country_id"].astype('category')
df_full["region_code"] = df_full["region_code"].astype('category')
df_full["gender"] =  df_full["gender"].astype('category')
df_full["age"] = df_full["age"].astype('int32')

## AGE

In [68]:
# Contar cuántos valores únicos de 'age' tiene cada cliente
estado_por_cliente_age = df_full.groupby('pk_cid')['age'].nunique()

# Identificar clientes con edades mixtas (múltiples valores en 'age')
clientes_mixtos_age = estado_por_cliente_age[estado_por_cliente_age > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_age = df_full[~df_full['pk_cid'].isin(clientes_mixtos_age)]

# Identificar clientes que tienen siempre la misma edad
clientes_misma_edad = df_no_mixtos_age.groupby('pk_cid')['age'].nunique()
clientes_misma_edad = clientes_misma_edad[clientes_misma_edad == 1].index

# Identificar clientes con valores NaN en 'age' (excluyendo clientes mixtos)
clientes_con_nan_age = df_no_mixtos_age[df_no_mixtos_age['age'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_misma_edad = len(clientes_misma_edad)
n_con_nan_age = len(clientes_con_nan_age)
n_mixtos_age = len(clientes_mixtos_age)

# Mostrar los resultados
print(f"Clientes que tienen siempre la misma edad (excluyendo mixtos): {n_misma_edad}")
print(f"Clientes con valores NaN en 'age' (excluyendo mixtos): {n_con_nan_age}")
print(f"Clientes con edades mixtas (diferentes valores de edad): {n_mixtos_age}")

Clientes que tienen siempre la misma edad (excluyendo mixtos): 114631
Clientes con valores NaN en 'age' (excluyendo mixtos): 0
Clientes con edades mixtas (diferentes valores de edad): 341742


In [69]:
n_total = n_misma_edad + n_con_nan_age + n_mixtos_age
n_total

456373

In [70]:
df_misma_edad = df_full[df_full['pk_cid'].isin(clientes_misma_edad)]
df_edad_mixta = df_full[df_full['pk_cid'].isin(clientes_mixtos_age)]

In [71]:
df_misma_edad["age"].describe()

count   1,200,493.00
mean           29.99
std            12.05
min             2.00
25%            22.00
50%            25.00
75%            35.00
max           102.00
Name: age, dtype: float64

In [72]:
df_edad_mixta["age"].describe()

count   4,762,431.00
mean           29.70
std            11.96
min             2.00
25%            22.00
50%            24.00
75%            34.00
max           105.00
Name: age, dtype: float64

aquí se tomará la decisión de trabajar con la última edad del cliente, también hay que considerar que hay valores entre 2 y 18 años que creo que no son adecuados para la ultilizacion de una aplicacion así y revisar tambien los valores mayores de 100 años.

(Adrián) Sí pueden ser datos reales, de hecho vamos a hacer algunas comprobaciones posteriores.

****************************************************************************************************************************************************************************************
                                                                               # ADRIAN                                                                                                 
****************************************************************************************************************************************************************************************


Vamos a buscar los clientes con edades anómalas, y que podrían ser objetos de una modificación. En este caso vamos a ver primero aquellos que se mantienen siempre con la misma edad, y además son menores

### Edad Constante

1. Identificar clientes con productos activos
Primero, filtramos el DataFrame clientes_misma_edad_menores para obtener los clientes que tienen algún producto activo (donde el valor es 1).

In [73]:
# Filtrar clientes que siempre tienen la misma edad y esa edad es menor a 18 años
clientes_misma_edad_menores = df_misma_edad[df_misma_edad['age'] < 18]

# Mostrar los resultados
clientes_misma_edad_menores

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
5520651,491473,2019-05-28,2019-04-27,KHM,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,V,2,N,NaN
5081423,541741,2019-04-28,2019-04-07,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,V,2,N,NaN
5520587,541741,2019-05-28,2019-04-07,KHK,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,V,2,N,NaN
1476977,617264,2018-07-28,2018-04-09,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,13,N,"286,573.44"
1843524,617264,2018-08-28,2018-04-09,KFC,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,13,N,"286,573.44"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5679711,1552805,2019-05-28,2019-05-26,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,V,2,N,NaN
5679415,1552858,2019-05-28,2019-05-26,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,38.00,V,11,N,NaN
5680043,1553226,2019-05-28,2019-05-30,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,41.00,H,9,N,NaN
5680169,1553282,2019-05-28,2019-05-30,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,43.00,H,4,N,NaN


In [74]:
# Lista de productos a analizar
productos = [
    'short_term_deposit', 'loans', 'mortgage', 'funds', 'securities',
    'long_term_deposit', 'em_account_pp', 'credit_card', 'payroll', 
    'pension_plan', 'payroll_account', 'emc_account', 'debit_card', 
    'em_account_p', 'em_acount'
]

# Contar el número de casos donde cada producto ha sido contratado (valor = 1)
conteo_productos = clientes_misma_edad_menores[productos].sum()

# Mostrar el resultado
conteo_productos

short_term_deposit   0.00
loans                0.00
mortgage             0.00
funds                0.00
securities           0.00
long_term_deposit    0.00
em_account_pp        0.00
credit_card          0.00
payroll              2.00
pension_plan         9.00
payroll_account      0.00
emc_account          0.00
debit_card           0.00
em_account_p         0.00
em_acount            3.00
dtype: float64

In [75]:
pd.set_option('display.max_columns', None)

# Filtrar los clientes que tienen productos activos
clientes_con_productos_activos = clientes_misma_edad_menores[clientes_misma_edad_menores[productos].sum(axis=1) > 0]

# Mostrar el resultado
clientes_con_productos_activos

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
5835610,1005293,2019-05-28,2015-02-16,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,28.00,V,6,N,"81,032.52"
5840803,1021968,2019-05-28,2015-07-04,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,28.00,H,13,N,"318,767.10"
5879905,1244964,2019-05-28,2017-02-24,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,28.00,V,5,N,NaN
107719,1256153,2018-01-28,2017-05-05,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,11.00,H,12,N,"156,534.36"
1687485,1256153,2018-07-28,2017-05-05,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,11.00,H,12,N,"156,534.36"
4071542,1256153,2019-01-28,2017-05-05,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,11.00,H,12,N,"156,534.36"
5551912,1345478,2019-05-28,2017-10-31,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,1.00,0,0,0,0,0,ES,28.00,V,4,N,"118,693.50"
1815226,1408784,2018-07-28,2018-07-20,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,22.00,V,17,N,"56,730.99"
1925092,1408784,2018-08-28,2018-07-20,KHQ,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,22.00,V,17,N,"56,730.99"
2458864,1408784,2018-09-28,2018-07-20,KHQ,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,22.00,V,17,N,"56,730.99"


2. Repetir el proceso para los active_customer

In [76]:
# Filtrar los clientes menores de 18 años que también son active_customer
clientes_menores_active = clientes_misma_edad_menores[clientes_misma_edad_menores['active_customer'] == 1]

# Contar cuántos clientes cumplen con ambas condiciones
cantidad_menores_active = clientes_menores_active.shape[0]

# Mostrar la cantidad
print(f"Cantidad de clientes menores de 18 años que son active_customer: {cantidad_menores_active}")

# Mostrar la lista de estos clientes
clientes_menores_active

Cantidad de clientes menores de 18 años que son active_customer: 6053


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
1476977,617264,2018-07-28,2018-04-09,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,13,N,"286,573.44"
83238,970787,2018-01-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,14,N,"234,170.37"
395797,970787,2018-02-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,14,N,"234,170.37"
635515,970787,2018-03-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,14,N,"234,170.37"
886240,970787,2018-04-28,2015-09-05,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,14,N,"234,170.37"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5681102,1552066,2019-05-28,2019-05-24,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,29.00,H,8,N,NaN
5681028,1552168,2019-05-28,2019-05-24,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,V,11,N,NaN
5680695,1552311,2019-05-28,2019-05-24,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,38.00,V,13,N,NaN
5680884,1552422,2019-05-28,2019-05-26,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,39.00,V,2,N,NaN


### Edad Mixta

En este apartado vamos a corregir errores de tipidficación en la variable Edad. Para ello, vamos a identificar en primer lugar casos en los que la edad tenga saltos mayores de 3 años (dado que el rango máximo de variación para un cliente debería ser este, pues la serie se compone de 3 años), y por otro lado, vamos a identificar aquellos casos en los que se produzca una variación negativa 

In [77]:
# Asegúrate de que df_edad_mixta esté ordenado por pk_cid y un índice de tiempo o la posición de las filas.
df_edad_mixta = df_edad_mixta.sort_values(by=['pk_cid', 'pk_partition'])  # Asumiendo que pk_partition representa el tiempo.

# Calcular la diferencia entre la primera y la última edad para cada usuario
df_edad_mixta['dif_edad'] = df_edad_mixta.groupby('pk_cid')['age'].transform(lambda x: x.iloc[-1] - x.iloc[0])

# Mostrar el resultado
df_edad_mixta[['pk_cid', 'age', 'dif_edad']].drop_duplicates()

,pk_cid,age,dif_edad
2171490,16502,57,1
2943633,16502,58,1
237929,17457,53,1
5134250,17457,54,1
1479558,17590,54,1
...,...,...,...
5683006,1548180,38,-1
5100662,1548181,34,1
5683005,1548181,35,1
5100664,1548186,42,-1


#### Edad Mixta. Variación > 3 años

In [78]:
df_edad_mixta_dif_mas3 = df_edad_mixta[df_edad_mixta['dif_edad'] > 3]

df_edad_mixta_dif_mas3["dif_edad"].describe()

count   1,093.00
mean       25.08
std        14.55
min         5.00
25%        14.00
50%        21.00
75%        31.00
max        67.00
Name: dif_edad, dtype: float64

In [79]:
df_edad_mixta[df_edad_mixta['dif_edad'] > 60]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,dif_edad
2169124,539069,2018-09-28,2018-09-21,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,2,N,NaN,67
2945655,539069,2018-10-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,2,N,NaN,67
2955191,539069,2018-11-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,46.00,V,69,N,NaN,67
3630152,539069,2018-12-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,69,N,NaN,67
4102655,539069,2019-01-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,46.00,V,69,N,NaN,67
4479766,539069,2019-02-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,46.00,V,69,N,NaN,67
4886596,539069,2019-03-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,69,N,NaN,67
5080982,539069,2019-04-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,69,N,NaN,67
5520571,539069,2019-05-28,2018-09-21,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,46.00,V,69,N,NaN,67
1324522,1308540,2018-06-28,2017-09-16,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,16.00,V,21,N,"116,896.11",61


In [80]:
df_edad_mixta[(df_edad_mixta['dif_edad'] > 3) & (df_edad_mixta['dif_edad'] < 6)]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,dif_edad
1813962,1405875,2018-07-28,2018-07-13,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,20,N,"116,051.58",5
1929750,1405875,2018-08-28,2018-07-13,KHM,1,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,24,N,"116,051.58",5
2466761,1405875,2018-09-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,24,N,"116,051.58",5
2733258,1405875,2018-10-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,24,N,"116,051.58",5
3177827,1405875,2018-11-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,25,N,"116,051.58",5
3776945,1405875,2018-12-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,25,N,"116,051.58",5
3959819,1405875,2019-01-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,25,N,"116,051.58",5
4415986,1405875,2019-02-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,25,N,"116,051.58",5
4676045,1405875,2019-03-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,25,N,"116,051.58",5
5315811,1405875,2019-04-28,2018-07-13,KHM,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,40.00,V,25,N,"116,051.58",5


En este caso, vamos a sustituir todos los valores de Age por el último valor conocido, ya que, a futuro lo más coherente será crear categorías de edad en las que se agrupen distintas décadas, por tanto, las variaciones internas de cada cliente van a ser practicamente irrelevantes (si un cliente pasa de 30-33 no es diferenciador), y además, se sobreentiende que los errores de tipificación se corrigen con el tiempo, por tanto, cuanto más reciente sea una entrada, más probable es que sea correcta.

In [81]:
# Paso 1: Filtrar los individuos que cumplen la condición dif_edad > 3
df_edad_mixta_dif_mas3 = df_edad_mixta[df_edad_mixta['dif_edad'] > 3]

# Paso 2: Obtener la última edad registrada (en la última pk_partition) para cada individuo
ultima_edad_por_individuo = df_edad_mixta_dif_mas3.groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'age'])

# Paso 3: Actualizar la edad en df_full para los individuos que cumplen la condición
def actualizar_edad(row):
    if row['pk_cid'] in ultima_edad_por_individuo.index:
        return ultima_edad_por_individuo[row['pk_cid']]
    else:
        return row['age']

# Aplicar la función para actualizar la edad en df_full
df_full['age'] = df_full.apply(actualizar_edad, axis=1)

# Mostrar el resultado
df_full.head()

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\1147662366.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ultima_edad_por_individuo = df_edad_mixta_dif_mas3.groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'age'])


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
1479563,15891,2018-07-28,2018-07-28,KAT,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,28.00,H,59,N,NaN
2168122,15891,2018-08-28,2018-07-28,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,59,N,NaN
2962973,16063,2018-11-28,2018-11-19,KAT,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,62,N,NaN
3628236,16063,2018-12-28,2018-11-19,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,62,N,NaN
4028169,16063,2019-01-28,2018-11-19,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,62,N,NaN


#### Edad Mixta. Variación Negativa

In [82]:
# Filtrar los casos donde la diferencia de edad es menor que 0
df_dif_edad_neg = df_edad_mixta[df_edad_mixta['dif_edad'] < 0]

df_dif_edad_neg["dif_edad"].describe()

count   94,401.00
mean        -1.01
std          0.42
min        -40.00
25%         -1.00
50%         -1.00
75%         -1.00
max         -1.00
Name: dif_edad, dtype: float64

In [83]:
# Visualizar los casos donde la diferencia de edad es menor que 0
df_dif_edad_neg

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,dif_edad
2943726,46732,2018-10-28,2018-10-08,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,71,N,NaN,-1
2977216,46732,2018-11-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,70,N,NaN,-1
3628313,46732,2018-12-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,70,N,NaN,-1
3839343,46732,2019-01-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,70,N,NaN,-1
4480561,46732,2019-02-28,2018-10-08,KFC,1,02 - PARTICULARES,0,0,0,0,0,1,0,0,0.00,0.00,0,0,0,0,1,ES,46.00,V,70,N,NaN,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5682969,1548156,2019-05-28,2019-04-29,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,50.00,V,62,N,NaN,-1
5100661,1548180,2019-04-28,2019-04-29,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,H,39,N,NaN,-1
5683006,1548180,2019-05-28,2019-04-29,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,28.00,H,38,N,NaN,-1
5100664,1548186,2019-04-28,2019-04-29,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,3.00,V,42,N,NaN,-1


In [84]:
df_dif_edad_neg[df_dif_edad_neg['dif_edad'] < -1]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,dif_edad
3033723,308667,2018-11-28,2018-11-16,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,50.00,V,65,N,NaN,-9
3627875,308667,2018-12-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,50.00,V,65,N,NaN,-9
4148494,308667,2019-01-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,50.00,V,55,N,NaN,-9
4481102,308667,2019-02-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,50.00,V,55,N,NaN,-9
4884600,308667,2019-03-28,2018-11-16,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,50.00,V,55,N,NaN,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5698512,1526523,2019-05-28,2019-01-22,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,ES,28.00,H,34,N,NaN,-10
4394326,1530367,2019-02-28,2019-02-05,NaN,0,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,32.00,V,86,N,NaN,-40
4720580,1530367,2019-03-28,2019-02-05,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,1.00,1.00,1,0,1,0,0,ES,32.00,V,86,N,NaN,-40
5084070,1530367,2019-04-28,2019-02-05,KHN,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,1.00,1.00,1,0,1,0,0,ES,32.00,V,46,N,NaN,-40


De igual forma vamos a sustituir todas las edades de los casos con variaciones negativas por la edad que figure en la última entrada de cada usuario

In [85]:
# Paso 1: Obtener la última edad registrada (en la última pk_partition) para cada individuo
ultima_edad_por_individuo_neg = df_dif_edad_neg.groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'age'])

# Paso 2: Actualizar la edad en df_full para los individuos que cumplen la condición
def actualizar_edad_neg(row):
    if row['pk_cid'] in ultima_edad_por_individuo_neg.index:
        return ultima_edad_por_individuo_neg[row['pk_cid']]
    else:
        return row['age']

# Aplicar la función para actualizar la edad en df_full
df_full['age'] = df_full.apply(actualizar_edad_neg, axis=1)

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\1033597339.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ultima_edad_por_individuo_neg = df_dif_edad_neg.groupby('pk_cid').apply(lambda x: x.loc[x['pk_partition'].idxmax(), 'age'])


## COUNTRY ID

In [86]:
# Contar cuántos valores únicos de 'country_id' tiene cada cliente
estado_por_cliente_country = df_full.groupby('pk_cid')['country_id'].nunique()

# Identificar clientes con 'country_id' mixtos (múltiples valores de 'country_id')
clientes_mixtos_country = estado_por_cliente_country[estado_por_cliente_country > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_country = df_full[~df_full['pk_cid'].isin(clientes_mixtos_country)]

# Identificar clientes que tienen siempre el mismo 'country_id'
clientes_mismo_country = df_no_mixtos_country.groupby('pk_cid')['country_id'].nunique()
clientes_mismo_country = clientes_mismo_country[clientes_mismo_country == 1].index

# Identificar clientes con valores NaN en 'country_id' (excluyendo clientes mixtos)
clientes_con_nan_country = df_no_mixtos_country[df_no_mixtos_country['country_id'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_mismo_country = len(clientes_mismo_country)
n_con_nan_country = len(clientes_con_nan_country)
n_mixtos_country = len(clientes_mixtos_country)

# Mostrar los resultados
print(f"Clientes que tienen siempre el mismo 'country_id' (excluyendo mixtos): {n_mismo_country}")
print(f"Clientes con valores NaN en 'country_id' (excluyendo mixtos): {n_con_nan_country}")
print(f"Clientes con 'country_id' mixtos (diferentes países): {n_mixtos_country}")

Clientes que tienen siempre el mismo 'country_id' (excluyendo mixtos): 456348
Clientes con valores NaN en 'country_id' (excluyendo mixtos): 0
Clientes con 'country_id' mixtos (diferentes países): 25


Los clientes con country id distintos como que antes vivian en un pais y ahora se han mudado para otro, no sé como tratar esto, si mantener el ultimo o el primero. (Adrián) Yo no los cambiaría, la variable indica país de residencia, por tanto, puede ser variable, y son muy pocos casos, no creo que supongan una distorsión de resultados

In [87]:
# Lista de valores únicos en la columna 'country_id'
df_full['country_id'].value_counts()

country_id
ES    5960672
GB        441
FR        225
DE        199
US        195
CH        194
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
DO         17
SN         17
SA         17
RU         17
QA         17
NO         17
CI         17
MR         17
LU         17
CM         17
GA         17
CO         17
ET         17
GT         17
SE         16
JM         11
PT         11
DJ         11
RO          9
HU          8
DZ          7
PE          4
Name: count, dtype: int64

In [88]:
# Filtrar el DataFrame para obtener las filas donde el país es mixto
df_full[df_full['pk_cid'].isin(clientes_mixtos_country)]


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
1843228,670953,2018-08-28,2018-08-11,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,8.00,V,39,N,"111,759.69"
2268552,670953,2018-09-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,8.00,V,39,N,"111,759.69"
2946158,670953,2018-10-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,8.00,V,39,N,"111,759.69"
2954597,670953,2018-11-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,8.00,V,39,N,"111,759.69"
3628803,670953,2018-12-28,2018-08-11,KFA,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,GB,NaN,V,39,N,"111,759.69"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730692,1519831,2019-03-28,2018-12-29,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,GB,NaN,H,26,N,NaN
5121965,1519831,2019-04-28,2018-12-29,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,GB,NaN,H,26,N,NaN
5686361,1519831,2019-05-28,2018-12-29,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,1,0,1,GB,NaN,H,26,N,NaN
5096824,1542679,2019-04-28,2019-03-31,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,ES,7.00,V,64,N,NaN


## DECEASED

In [89]:
import pandas as pd

# Contar cuántos valores únicos de 'deceased' tiene cada cliente
estado_por_cliente_deceased = df_full.groupby('pk_cid')['deceased'].nunique()

# Identificar clientes con estados mixtos (N y S o con NaN incluido)
clientes_mixtos_deceased = estado_por_cliente_deceased[estado_por_cliente_deceased > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_deceased = df_full[~df_full['pk_cid'].isin(clientes_mixtos_deceased)]

# Identificar clientes que son siempre 'N'
clientes_siempre_n = df_no_mixtos_deceased[df_no_mixtos_deceased['deceased'] == 'N']['pk_cid'].unique()

# Identificar clientes que son siempre 'S'
clientes_siempre_s = df_no_mixtos_deceased[df_no_mixtos_deceased['deceased'] == 'S']['pk_cid'].unique()

# Identificar clientes con valores NaN en 'deceased' (excluyendo clientes mixtos)
clientes_con_nan_deceased = df_no_mixtos_deceased[df_no_mixtos_deceased['deceased'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_siempre_n = len(clientes_siempre_n)
n_siempre_s = len(clientes_siempre_s)
n_con_nan_deceased = len(clientes_con_nan_deceased)
n_mixtos_deceased = len(clientes_mixtos_deceased)

# Mostrar los resultados
print(f"Clientes que son siempre 'N' en 'deceased' (excluyendo mixtos): {n_siempre_n}")
print(f"Clientes que son siempre 'S' en 'deceased' (excluyendo mixtos): {n_siempre_s}")
print(f"Clientes con valores NaN en 'deceased' (excluyendo mixtos): {n_con_nan_deceased}")
print(f"Clientes con estados mixtos en 'deceased' (N, S o NaN): {n_mixtos_deceased}")

Clientes que son siempre 'N' en 'deceased' (excluyendo mixtos): 456244
Clientes que son siempre 'S' en 'deceased' (excluyendo mixtos): 55
Clientes con valores NaN en 'deceased' (excluyendo mixtos): 0
Clientes con estados mixtos en 'deceased' (N, S o NaN): 74


In [90]:
n_total = n_siempre_n + n_siempre_s + n_con_nan_deceased + n_mixtos_deceased
n_total

456373

hay 55 clientes muertos desde la primera ingesta de datos incluido el entry_date, serán errores o se tendrán que analizar mas a fondo así como los que cambian

## GENDER

In [91]:
# se crea una categoria Unknown para los valores nulos
df_full["gender"] = df_full["gender"].cat.add_categories(["Unknown"]).fillna("Unknown")

df_full.groupby("gender")["pk_cid"].nunique().apply(lambda x: x/df_full["pk_cid"].nunique()*100)

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\2251678930.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("gender")["pk_cid"].nunique().apply(lambda x: x/df_full["pk_cid"].nunique()*100)


gender
H         51.24
V         48.76
Unknown    0.00
Name: pk_cid, dtype: float64

In [92]:
df_full.groupby("gender")["pk_cid"].nunique()

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\204718755.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("gender")["pk_cid"].nunique()


gender
H          233862
V          222550
Unknown         2
Name: pk_cid, dtype: int64

In [93]:
df_full.groupby("gender")["pk_cid"].nunique().sum()
# se observa que hay 41 valores más que la cantidad de clientes únicos
# lo que representa que hay clientes con más de un género asignado

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\3905795626.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_full.groupby("gender")["pk_cid"].nunique().sum()


456414

### Análisis Géneros

In [94]:

# Contar cuántos valores únicos tiene cada cliente en la columna gender
estado_por_cliente_gender = df_full.groupby('pk_cid')['gender'].agg(['nunique', 'count'])

# Identificar clientes con géneros mixtos (H y F o Unknown)
clientes_mixtos_gender = estado_por_cliente_gender[estado_por_cliente_gender['nunique'] > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_gender = df_full[~df_full['pk_cid'].isin(clientes_mixtos_gender)]

# Identificar clientes que son siempre 'H'
clientes_siempre_H = df_no_mixtos_gender[df_no_mixtos_gender['gender'] == 'H']['pk_cid'].unique()

# Identificar clientes que son siempre 'V'
clientes_siempre_V = df_no_mixtos_gender[df_no_mixtos_gender['gender'] == 'V']['pk_cid'].unique()

# Identificar clientes "Unknown" en gender (excluyendo clientes mixtos)
clientes_siempre_Unknown = df_no_mixtos_gender[df_no_mixtos_gender['gender'] == "Unknown"]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_siempre_H = len(clientes_siempre_H)
n_siempre_V = len(clientes_siempre_V)
n_siempre_Unknown = len(clientes_siempre_Unknown)
n_mixtos_gender = len(clientes_mixtos_gender)

# Mostrar los resultados
print(f"Clientes que son siempre 'H' en 'gender' (excluyendo mixtos): {n_siempre_H}")
print(f"Clientes que son siempre 'V' en 'gender' (excluyendo mixtos): {n_siempre_V}")
print(f"Clientes que son todo 'Unknown' (excluyendo mixtos): {n_siempre_Unknown}")
print(f"Clientes con géneros mixtos (H y V o Unknown): {n_mixtos_gender}")
print(f"Total de clientes en el dataset: {n_siempre_H + n_siempre_V + n_siempre_Unknown + n_mixtos_gender}")


Clientes que son siempre 'H' en 'gender' (excluyendo mixtos): 233821
Clientes que son siempre 'V' en 'gender' (excluyendo mixtos): 222509
Clientes que son todo 'Unknown' (excluyendo mixtos): 2
Clientes con géneros mixtos (H y V o Unknown): 41
Total de clientes en el dataset: 456373


In [95]:
# Poroporciones
n_total = n_siempre_H + n_siempre_V + n_siempre_Unknown + n_mixtos_gender
p_siempre_H = n_siempre_H / n_total * 100
p_siempre_V = n_siempre_V / n_total * 100
p_siempre_Unknown = n_siempre_Unknown / n_total * 100
p_mixtos_gender = n_mixtos_gender / n_total * 100

# Mostrar los resultados
print(f"Clientes que son siempre 'H': {p_siempre_H:.2f}%")
print(f"Clientes que son siempre 'V': {p_siempre_V:.2f}%")
print(f"Clientes que son todo 'Unknown': {p_siempre_Unknown:.5f}%")
print(f"Clientes con géneros mixtos (H y V o Unknown): {p_mixtos_gender:.3f}%")

Clientes que son siempre 'H': 51.23%
Clientes que son siempre 'V': 48.76%
Clientes que son todo 'Unknown': 0.00044%
Clientes con géneros mixtos (H y V o Unknown): 0.009%


### Género unknown constante

In [96]:
# Clientes donde el valor de 'gender' es 'Unknown'
df_full[df_full['gender'] == "Unknown"]

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
2168807,216507,2018-08-28,2018-08-17,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,Unknown,72,N,"104,296.62"
2172127,216507,2018-09-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,Unknown,72,N,"104,296.62"
2943933,216507,2018-10-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,ES,28.00,Unknown,72,N,"104,296.62"
3033652,216507,2018-11-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,72,N,"104,296.62"
3627544,216507,2018-12-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,72,N,"104,296.62"
3839882,216507,2019-01-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,73,N,"104,296.62"
4481238,216507,2019-02-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,72,N,"104,296.62"
4875861,216507,2019-03-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,73,N,"104,296.62"
5133231,216507,2019-04-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,73,N,"104,296.62"
5523191,216507,2019-05-28,2018-08-17,KHO,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,1,0,0,0,ES,28.00,Unknown,73,N,"104,296.62"


In [97]:
# Reemplazar el valor 'Unknown' por 'H' en la columna 'gender'
df_full.loc[df_full['gender'] == "Unknown", 'gender'] = "H"

Vamos a imputar los Unknown y los vamos a cambiar por Hombre, ya que, ambos individuos superan los 70 años (sería extraño que se identiffiquen como género fluido y la variable H es la que presenta mayor proporción, por tanto lo imputaremos a esta)

### Género Unknown Mixto

In [98]:
# Aplicar el método ffill y bfill para imputar Unknown dentro de cada grupo
df_full['gender'] = df_full.groupby('pk_cid')['gender'].transform(lambda x: x.ffill().bfill())

# Verificar si hay todavía valores Unknown después de la imputación
unknown_after = df_full[df_full['gender'] == "Unknown"]

print(f"Número de valores 'Unknown' después de la imputación: {len(unknown_after)}")


Número de valores 'Unknown' después de la imputación: 0


## REGION_CODE


In [99]:
# valores nulos en region_code ponerlos como Unknown
df_full["region_code"] = df_full["region_code"].cat.add_categories(["Unknown"]).fillna("Unknown")


In [100]:

import pandas as pd

# Contar cuántos valores únicos de 'region_code' tiene cada cliente
estado_por_cliente_region = df_full.groupby('pk_cid')['region_code'].nunique()

# Identificar clientes con múltiples códigos de región (cambios de región)
clientes_mixtos_region = estado_por_cliente_region[estado_por_cliente_region > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_region = df_full[~df_full['pk_cid'].isin(clientes_mixtos_region)]

# Identificar clientes que tienen siempre el mismo código de región
clientes_mismo_region = df_no_mixtos_region.groupby('pk_cid')['region_code'].nunique()
clientes_mismo_region = clientes_mismo_region[clientes_mismo_region == 1].index

# Identificar clientes con valores NaN en 'region_code' (excluyendo clientes mixtos)
clientes_con_nan_region = df_no_mixtos_region[df_no_mixtos_region['region_code'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_mismo_region = len(clientes_mismo_region)
n_con_nan_region = len(clientes_con_nan_region)
n_mixtos_region = len(clientes_mixtos_region)

# Mostrar los resultados
print(f"Clientes que tienen siempre el mismo código de región (excluyendo mixtos): {n_mismo_region}")
print(f"Clientes con valores NaN en 'region_code' (excluyendo mixtos): {n_con_nan_region}")
print(f"Clientes con códigos de región mixtos (diferentes códigos): {n_mixtos_region}")

Clientes que tienen siempre el mismo código de región (excluyendo mixtos): 454305
Clientes con valores NaN en 'region_code' (excluyendo mixtos): 0
Clientes con códigos de región mixtos (diferentes códigos): 2068


In [101]:
# se comprueba que la suma de los clientes en cada región es igual a la cantidad de clientes únicos
n_total = n_mismo_region + n_con_nan_region + n_mixtos_region
n_total 

456373

In [102]:
df_full[df_full["region_code"] == 'Unknown']

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
240946,130419,2018-02-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.00,0.00,0,0,0,0,0,US,Unknown,V,53,N,NaN
483522,130419,2018-03-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,US,Unknown,V,53,N,NaN
728777,130419,2018-04-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.00,0.00,0,0,0,0,0,US,Unknown,V,53,N,NaN
1223095,130419,2018-05-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,US,Unknown,V,53,N,NaN
1479139,130419,2018-07-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,US,Unknown,V,53,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5083649,1530982,2019-04-28,2019-02-09,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,GB,Unknown,V,33,N,NaN
5692590,1530982,2019-05-28,2019-02-09,KAT,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,1.00,1.00,1,0,0,0,0,GB,Unknown,V,33,N,NaN
5672371,1542679,2019-05-28,2019-03-31,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,US,Unknown,V,64,N,NaN
5097292,1544346,2019-04-28,2019-04-08,NaN,1,NaN,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,FR,Unknown,H,46,N,NaN


In [103]:
# Filtra el DataFrame según las condiciones especificadas
filtered_df = df_full[(df_full["region_code"] == 'Unknown') & (df_full["country_id"] == 'ES')]

filtered_df

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary
215020,1234433,2018-01-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
276408,1234433,2018-02-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,1,0,1,0,0,ES,Unknown,V,45,N,NaN
517930,1234433,2018-03-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
760234,1234433,2018-04-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
1190279,1234433,2018-05-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
1442145,1234433,2018-06-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
1744982,1234433,2018-07-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
2113491,1234433,2018-08-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,1.00,1.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
2256273,1234433,2018-09-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.00,0.00,0,0,1,0,0,ES,Unknown,V,45,N,NaN
2864559,1234433,2018-10-28,2016-12-27,KFC,1,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.00,0.00,0,0,0,0,0,ES,Unknown,V,45,N,NaN


Vamos a sustituir region_code por 28 en estos casos, al ser Madrid la región más repetida

In [105]:
# Verificar las categorías actuales
print("Categorías actuales:", df_full['region_code'].cat.categories)

# Agregar nuevas categorías
df_full['region_code'] = df_full['region_code'].cat.add_categories(['28', '00'])

# Aplicar las actualizaciones
df_full.loc[(df_full["region_code"] == 'Unknown') & (df_full["country_id"] == 'ES'), "region_code"] = '28'
df_full.loc[(df_full["region_code"] == 'Unknown') & (df_full["country_id"] != 'ES'), "region_code"] = '00'

# Verifica si quedan valores 'Unknown'
unknown_rows = df_full[df_full["region_code"] == 'Unknown']

unknown_rows

Categorías actuales: Index([      1.0,       2.0,       3.0,       4.0,       5.0,       6.0,
             7.0,       8.0,       9.0,      10.0,      11.0,      12.0,
            13.0,      14.0,      15.0,      16.0,      17.0,      18.0,
            19.0,      20.0,      21.0,      22.0,      23.0,      24.0,
            25.0,      26.0,      27.0,      28.0,      29.0,      30.0,
            31.0,      32.0,      33.0,      34.0,      35.0,      36.0,
            37.0,      38.0,      39.0,      40.0,      41.0,      42.0,
            43.0,      44.0,      45.0,      46.0,      47.0,      48.0,
            49.0,      50.0,      51.0,      52.0, 'Unknown'],
      dtype='object')


,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary


## SALARY

In [106]:
# Contar cuántos valores únicos de 'salary' tiene cada cliente
estado_por_cliente_salary = df_full.groupby('pk_cid')['salary'].nunique()

# Identificar clientes con múltiples valores de salario (cambios en el salario)
clientes_mixtos_salary = estado_por_cliente_salary[estado_por_cliente_salary > 1].index

# Excluir los clientes mixtos del análisis de las otras categorías
df_no_mixtos_salary = df_full[~df_full['pk_cid'].isin(clientes_mixtos_salary)]

# Identificar clientes que tienen siempre el mismo salario
clientes_mismo_salary = df_no_mixtos_salary.groupby('pk_cid')['salary'].nunique()
clientes_mismo_salary = clientes_mismo_salary[clientes_mismo_salary == 1].index

# Identificar clientes con valores NaN en 'salary' (excluyendo clientes mixtos)
clientes_con_nan_salary = df_no_mixtos_salary[df_no_mixtos_salary['salary'].isna()]['pk_cid'].unique()

# Contar el número de clientes en cada categoría (excluyendo mixtos)
n_mismo_salary = len(clientes_mismo_salary)
n_con_nan_salary = len(clientes_con_nan_salary)
n_mixtos_salary = len(clientes_mixtos_salary)

# Mostrar los resultados
print(f"Clientes que tienen siempre el mismo salario: {n_mismo_salary}")
print(f"Clientes con valores NaN en 'salary' en todas las ingestas: {n_con_nan_salary}")
print(f"Clientes con valores de salario mixtos, varían con la ingesta de datos (diferentes valores): {n_mixtos_salary}")

Clientes que tienen siempre el mismo salario: 299443
Clientes con valores NaN en 'salary' en todas las ingestas: 156930
Clientes con valores de salario mixtos, varían con la ingesta de datos (diferentes valores): 0


In [107]:
df_full["salary"].describe()

count    4,450,821.00
mean       115,816.72
std        199,551.89
min          1,202.73
25%         61,500.63
50%         88,654.65
75%        131,669.91
max     28,894,395.51
Name: salary, dtype: float64

In [108]:
# Filtrar los registros donde 'salary' es NaN
casos_salary_nan = df_full[df_full['salary'].isna()]

casos_salary_nan["country_id"].value_counts()

country_id
ES    1509888
GB        435
FR        225
DE        199
US        195
CH        188
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         29
CN         28
CA         18
DO         17
SN         17
SA         17
RU         17
QA         17
NO         17
CI         17
MR         17
LU         17
CM         17
GA         17
CO         17
ET         17
GT         17
SE         16
JM         11
PT         11
DJ         11
HU          8
DZ          0
PE          0
RO          0
Name: count, dtype: int64

Como tenemos valores nulos de practicamente todos los países, se me ocurre aplicar el salario mediano pero atendiendo al grupo de edad del sujeto, país de residencia y actividad comercial.  Por tanto, en primer lugar vamos a segmentar en grupos de edad. La segmentación de edades más utilizada suele ser la siguiente:

Menores: eGrupo compuesto por personas menores de dieciocho años

Jóvenes: El grupo de jóvenes son las personas mayores a dieciocho años y menores de veinticinco años.

Adultos jóvenes: Este segmento incluye personas mayores de veinticinco años y menores de cuarenta años.

Adultos: Son las personas mayores de cuarenta años y menores a los cincuenta y cinco años.

Adultos mayores: Encontramos en este grupo a las personas mayores de cincuenta y cinco años y menores de sesenta y cinco.

Ancianos: Son las personas con más sesenta y cinco años y menos de setenta y cinco años.

Longevos: Son todas las personas de setenta y cinco años en adelante.

In [109]:
# Definir función para categorizar según grupos de edad
def categorizar_edad(edad):
    if edad < 18:
        return 'Menores'
    elif 18 <= edad < 25:
        return 'Jóvenes'
    elif 25 <= edad < 40:
        return 'Adultos jóvenes'
    elif 40 <= edad < 55:
        return 'Adultos'
    elif 55 <= edad < 65:
        return 'Adultos mayores'
    elif 65 <= edad < 75:
        return 'Ancianos'
    else:
        return 'Longevos'

# Crear nueva columna 'grupo_edad' en el DataFrame
df_full['grupo_edad'] = df_full['age'].apply(categorizar_edad)

Ahora que tenemos los grupos de edad definidos, podemos proceder a imputar los valores nulos de salary usando la mediana del salario dentro de cada segmento.

In [110]:
# Reemplazar los valores NaN en la columna 'Segment' con '03 - UNIVERSITARIO'
df_full['segment'] = df_full['segment'].fillna('03 - UNIVERSITARIO')

# Calcular la mediana de 'salary' para cada combinación de país, grupo de edad y actividad comercial
mediana_salary = df_full.groupby(['country_id', 'grupo_edad', 'segment'])['salary'].median()

# Imputar los valores nulos de 'salary' con la mediana correspondiente
df_full['salary'] = df_full.apply(
    lambda row: mediana_salary[row['country_id'], row['grupo_edad'], row['segment']]
    if pd.isna(row['salary']) else row['salary'], axis=1
)

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\872123607.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  mediana_salary = df_full.groupby(['country_id', 'grupo_edad', 'segment'])['salary'].median()


In [111]:
df_full["salary"].describe()

count    5,961,175.00
mean       109,021.26
std        172,847.69
min          1,202.73
25%         70,173.33
50%         88,419.99
75%        114,538.62
max     28,894,395.51
Name: salary, dtype: float64

Categorizar edades:

La función categorizar_edad toma la edad y la clasifica en uno de los grupos definidos.

Luego, apply se utiliza para aplicar esta función a la columna age, creando una nueva columna grupo_edad en el DataFrame.

Calcular mediana:

groupby se utiliza para agrupar los datos por país (country_id), grupo de edad (grupo_edad), y actividad comercial (pk_partition).

La mediana del salario para cada grupo se calcula y se almacena en mediana_salary.

Imputación:

Para cada fila, si el salario es nulo (NaN), se reemplaza con la mediana correspondiente calculada en el paso anterior.

In [112]:
# Filtrar los registros donde 'salary' es NaN
casos_salary_nan = df_full[df_full['salary'].isna()]

# Mostrar los casos con 'salary' como NaN
casos_salary_nan

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment,short_term_deposit,loans,mortgage,funds,securities,long_term_deposit,em_account_pp,credit_card,payroll,pension_plan,payroll_account,emc_account,debit_card,em_account_p,em_acount,country_id,region_code,gender,age,deceased,salary,grupo_edad
240946,130419,2018-02-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.00,0.00,0,0,0,0,0,US,00,V,53,N,NaN,Adultos
483522,130419,2018-03-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,US,00,V,53,N,NaN,Adultos
728777,130419,2018-04-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,1,0.00,0.00,0,0,0,0,0,US,00,V,53,N,NaN,Adultos
1223095,130419,2018-05-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,US,00,V,53,N,NaN,Adultos
1479139,130419,2018-07-28,2017-12-31,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,US,00,V,53,N,NaN,Adultos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5698373,1528243,2019-05-28,2019-01-29,KAT,0,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,SE,00,V,32,N,NaN,Adultos jóvenes
4388560,1530982,2019-02-28,2019-02-09,NaN,0,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,GB,00,V,33,N,NaN,Adultos jóvenes
5672371,1542679,2019-05-28,2019-03-31,KHM,1,02 - PARTICULARES,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,US,00,V,64,N,NaN,Adultos mayores
5097292,1544346,2019-04-28,2019-04-08,NaN,1,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,1,FR,00,H,46,N,NaN,Adultos


Comprobamos que siguen qeudando casos con salary = Nan, aunqueson bastante escasos 1749 filas, que se corresponden con 130 individuos, puede ser que para estos no haya grupos de referencia que cumplan con las mismas características establecidas. Como sigue habiendo datos con Salary Nan, vamos a rellenarlos con el salario mediano del grupo_edad y el country_id

In [113]:
# Calcular la mediana del salario por grupo_edad y country_id
median_salary = df_full.groupby(['grupo_edad', 'country_id'])['salary'].median().reset_index()

# Renombrar la columna de salario mediano para mayor claridad
median_salary.rename(columns={'salary': 'median_salary'}, inplace=True)

# Combinar la mediana de salario con el DataFrame original
df_full = pd.merge(df_full, median_salary, on=['grupo_edad', 'country_id'], how='left')

# Rellenar los NaN en la columna salary con la mediana correspondiente
df_full['salary'].fillna(df_full['median_salary'], inplace=True)

# Eliminar la columna temporal de salario mediano si ya no es necesaria
df_full.drop(columns=['median_salary'], inplace=True)

# Verificar que no queden valores NaN en salary
nan_count = df_full['salary'].isna().sum()
print(f"Número de valores NaN restantes en 'salary': {nan_count}")

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\4288050092.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_salary = df_full.groupby(['grupo_edad', 'country_id'])['salary'].median().reset_index()
C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\4288050092.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full['salary'].

Número de valores NaN restantes en 'salary': 1666


Por último, solo nos quedaremos con la media de country_id al ser una variable que intutivamente más influencia tendrá en los cambios de salario

In [114]:
# Calcular la mediana del salario por country_id
median_salary = df_full.groupby(['country_id'])['salary'].median().reset_index()

# Renombrar la columna de salario mediano para mayor claridad
median_salary.rename(columns={'salary': 'median_salary'}, inplace=True)

# Combinar la mediana de salario con el DataFrame original
df_full = pd.merge(df_full, median_salary, on=['country_id'], how='left')

# Rellenar los NaN en la columna salary con la mediana correspondiente
df_full['salary'].fillna(df_full['median_salary'], inplace=True)

# Eliminar la columna temporal de salario mediano si ya no es necesaria
df_full.drop(columns=['median_salary'], inplace=True)

# Verificar que no queden valores NaN en salary
nan_count = df_full['salary'].isna().sum()
print(f"Número de valores NaN restantes en 'salary': {nan_count}")

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\1632973577.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  median_salary = df_full.groupby(['country_id'])['salary'].median().reset_index()
C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\1632973577.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full['salary'].fillna(df_full

Número de valores NaN restantes en 'salary': 1545


In [115]:
# Calcular la mediana del salario por grupo_edad
median_salary = df_full.groupby(['grupo_edad'])['salary'].median().reset_index()

# Renombrar la columna de salario mediano para mayor claridad
median_salary.rename(columns={'salary': 'median_salary'}, inplace=True)

# Combinar la mediana de salario con el DataFrame original
df_full = pd.merge(df_full, median_salary, on=['grupo_edad'], how='left')

# Rellenar los NaN en la columna salary con la mediana correspondiente
df_full['salary'].fillna(df_full['median_salary'], inplace=True)

# Eliminar la columna temporal de salario mediano si ya no es necesaria
df_full.drop(columns=['median_salary'], inplace=True)

# Verificar que no queden valores NaN en salary
nan_count = df_full['salary'].isna().sum()
print(f"Número de valores NaN restantes en 'salary': {nan_count}")

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\1886888188.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_full['salary'].fillna(df_full['median_salary'], inplace=True)


Número de valores NaN restantes en 'salary': 0


In [116]:
# Crear una lista de las columnas que deseas eliminar
columnas_a_eliminar = [
    'pk_cid', 'pk_partition', 'entry_date', 'entry_channel', 'active_customer', 
    'segment', 'short_term_deposit', 'loans', 'mortgage', 'funds', 
    'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 
    'payroll', 'pension_plan', 'payroll_account', 'emc_account', 
    'debit_card', 'em_account_p', 'em_acount'
]

# Crear una copia del DataFrame original y eliminar las columnas especificadas
sociodemographic_df_adrian = df_full.copy()
sociodemographic_df_adrian = sociodemographic_df_adrian.drop(columns=columnas_a_eliminar)

# Verificar las primeras filas del nuevo DataFrame para asegurarse de que las columnas se eliminaron correctamente
sociodemographic_df_adrian.head()

,country_id,region_code,gender,age,deceased,salary,grupo_edad
0,ES,28.00,H,59,N,"100,781.46",Adultos mayores
1,ES,28.00,H,59,N,"99,034.08",Adultos mayores
2,ES,28.00,H,62,N,"100,781.46",Adultos mayores
3,ES,28.00,H,62,N,"99,034.08",Adultos mayores
4,ES,28.00,H,62,N,"99,034.08",Adultos mayores


In [117]:
# Crear una lista de las columnas que deseas eliminar
columnas_a_eliminar = [
    'entry_date', 'entry_channel', 'active_customer', 
    'segment', 'short_term_deposit', 'loans', 'mortgage', 'funds', 
    'securities', 'long_term_deposit', 'em_account_pp', 'credit_card', 
    'payroll', 'pension_plan', 'payroll_account', 'emc_account', 
    'debit_card', 'em_account_p', 'em_acount'
]

# Crear una copia del DataFrame original y eliminar las columnas especificadas
sociodemographic_df_adrian = df_full.copy()
sociodemographic_df_adrian = sociodemographic_df_adrian.drop(columns=columnas_a_eliminar)

# Verificar las primeras filas del nuevo DataFrame para asegurarse de que las columnas se eliminaron correctamente
sociodemographic_df_adrian.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary,grupo_edad
0,15891,2018-07-28,ES,28.00,H,59,N,"100,781.46",Adultos mayores
1,15891,2018-08-28,ES,28.00,H,59,N,"99,034.08",Adultos mayores
2,16063,2018-11-28,ES,28.00,H,62,N,"100,781.46",Adultos mayores
3,16063,2018-12-28,ES,28.00,H,62,N,"99,034.08",Adultos mayores
4,16063,2019-01-28,ES,28.00,H,62,N,"99,034.08",Adultos mayores


In [118]:
# Supongamos que tu DataFrame se llama df_full
df_full.to_csv('sociodemographic_df_adrian.csv', index=False)

In [119]:
# Columnas de interés
columns_of_interest = ['pk_cid', 'pk_partition', 'country_id', 'region_code', 'gender', 'age', 'deceased', 'salary', 'grupo_edad']

# Comprobar valores NaN
nan_check = sociodemographic_df_adrian[columns_of_interest].isna().sum()

# Comprobar valores 'Unknown'
unknown_check = sociodemographic_df_adrian[columns_of_interest].applymap(lambda x: x == 'Unknown').sum()

# Mostrar resultados
print("Número de valores NaN por columna:")
print(nan_check)

print("\nNúmero de valores 'Unknown' por columna:")
print(unknown_check)

C:\Users\Bobby\AppData\Local\Temp\ipykernel_23424\1090408245.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  unknown_check = sociodemographic_df_adrian[columns_of_interest].applymap(lambda x: x == 'Unknown').sum()


Número de valores NaN por columna:
pk_cid          0
pk_partition    0
country_id      0
region_code     0
gender          0
age             0
deceased        0
salary          0
grupo_edad      0
dtype: int64

Número de valores 'Unknown' por columna:
pk_cid          0
pk_partition    0
country_id      0
region_code     0
gender          0
age             0
deceased        0
salary          0
grupo_edad      0
dtype: int64
